## Script Python Algorithme de recommandation de films pour l'apprentissage des langues

**Auteur :** _Léna Rebours_  
**Date :** _07/11/2025_ 


### Objectif du script

Ce script Python constitue le **MVP** de mon système de recommandation de contenus culturels pour l’apprentissage des langues. Pour avoir une liste de recommandations, il suffit de faire tourner le script et l'algorithme vous posera quelques questions sur vos préférences.

Pour cette première version, le focus est volontairement restreint à :

- un seul type de contenu : **les films** ;
- un seul jeu de données : **les films de 1950 à 1959** (`films_1950_1959_merged.csv`) ;
- un fonctionnement simple mais concret : **filtrage déclaratif** à partir des préférences de l’utilisateur.

## 1. Librairies

In [1]:
import pandas as pd
import ast

## 2. Algorithme de recommandation

### 2.1. Chargement des données

### Nettoyage des données et ligne bizarre

In [3]:
import pandas as pd
import re
import csv

IN = "films_TMDB_all_reordered.csv"
OUT = "films_TMDB_all_reordered_clean.csv"

CHUNK = 100_000   # parfait pour ~900k lignes

def clean_cell(x):
    if pd.isna(x):
        return x
    s = str(x)
    s = re.sub(r"[\r\n\t]+", " ", s)   # enlève retours ligne / tabs
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip()

first = True
for chunk in pd.read_csv(IN, chunksize=CHUNK, low_memory=False):
    # nettoyer uniquement les colonnes texte
    for c in chunk.select_dtypes(include="object").columns:
        chunk[c] = chunk[c].map(clean_cell)

    chunk.to_csv(
        OUT,
        mode="w" if first else "a",
        index=False,
        header=first,
        encoding="utf-8-sig",
        quoting=csv.QUOTE_MINIMAL,
        lineterminator="\n"
    )
    first = False

print("CSV nettoyé :", OUT)


CSV nettoyé : films_TMDB_all_reordered_clean.csv


In [5]:
CSV_PATH = "films_TMDB_all_reordered_clean.csv"
df = pd.read_csv(CSV_PATH)

### Normalement le  csv sortant est publiable 

### 2.2. Préparation / nettoyage de quelques colonnes

In [6]:
# On s'assure que la colonne 'popularity' est bien numérique
df["popularity"] = pd.to_numeric(df["popularity"], errors="coerce")

# On s'assure que 'vote_average' est numérique aussi 
if "vote_average" in df.columns:
    df["vote_average"] = pd.to_numeric(df["vote_average"], errors="coerce")

# On s'assure que 'runtime' est numérique 
if "runtime" in df.columns:
    df["runtime"] = pd.to_numeric(df["runtime"], errors="coerce")

# Fonction utilitaire pour extraire la liste des genres à partir de la colonne 'genres'
def extract_genre_names(genres_cell):
    if pd.isna(genres_cell):
        return []

    # Si c'est déjà une liste, on utilise directement
    if isinstance(genres_cell, list):
        items = genres_cell
    else:
        try:
            items = ast.literal_eval(genres_cell)
        except (ValueError, SyntaxError):
            return []

    # On récupère les noms de genres, en minuscules pour faciliter les comparaisons
    names = []
    for g in items:
        if isinstance(g, dict) and "name" in g:
            names.append(g["name"].strip().lower())
        elif isinstance(g, str):
            names.append(g.strip().lower())
    return names

# On crée une nouvelle colonne avec la liste de genres normalisée
df["genre_list"] = df["genres"].apply(extract_genre_names)

### 2.3. Fonction principale de recommandation

In [7]:
def recommend_movies(df, top_n=20):
    """
    Interagit avec l'utilisateur pour filtrer les films selon :
    - une langue
    - un ou plusieurs genres souhaités
    - éventuellement d'autres critères simples
    Puis renvoie une liste de recommandations triées par popularité décroissante.
    """

    # -----------------------------
    # Étape 1 : choix de la langue
    # -----------------------------

    # On récupère la liste des langues disponibles (colonne 'original_language')
    if "original_language" not in df.columns:
        print("La colonne 'original_language' est manquante dans le dataset.")
        return

    languages = sorted(df["original_language"].dropna().unique())
    print("Langues disponibles dans le dataset :")
    print(", ".join(languages))

    user_lang = input("\nChoisis une langue (code ISO, ex: 'en', 'fr', 'it', 'ja') : ").strip().lower()

    # On filtre sur la langue choisie
    lang_df = df[df["original_language"].str.lower() == user_lang].copy()

    if lang_df.empty:
        print(f"\nAucun film trouvé pour la langue '{user_lang}'.")
        return

    print(f"\nNombre de films trouvés pour la langue '{user_lang}': {len(lang_df)}")

    # ----------------------------------------
    # Étape 2 : proposition des genres dispo
    # ----------------------------------------

    # On récupère tous les genres présents dans ce sous-ensemble
    all_genres = set()
    for genres in lang_df["genre_list"]:
        all_genres.update(genres)

    if not all_genres:
        print("\nAucun genre exploitable trouvé pour cette langue.")
        return

    sorted_genres = sorted(all_genres)
    print("\nGenres disponibles pour cette langue :")
    print(", ".join(sorted_genres))

    # L'utilisateur peut saisir plusieurs genres séparés par des virgules
    user_genres_input = input(
        "\nEntre un ou plusieurs genres souhaités (séparés par des virgules, ex: drama, comedy, documentary) : "
    ).strip().lower()

    # On transforme la saisie en liste nettoyée
    user_genres = [g.strip() for g in user_genres_input.split(",") if g.strip()]

    # Filtrage sur les genres : on garde les films qui contiennent AU MOINS un des genres choisis
    if user_genres:
        def match_genres(genre_list):
            return any(ug in genre_list for ug in user_genres)

        genre_filtered_df = lang_df[lang_df["genre_list"].apply(match_genres)].copy()
    else:
        # Si aucun genre n'est saisi, on garde tout (mais on prévient)
        print("\nAucun genre spécifié, on garde tous les genres pour cette langue.")
        genre_filtered_df = lang_df.copy()

    if genre_filtered_df.empty:
        print("\nAucun film ne correspond aux genres sélectionnés.")
        return

    print(f"\nFilms après filtre sur les genres : {len(genre_filtered_df)}")

    # ---------------------------------------------------
    # Étape 3 : critères additionnels simples
    # ---------------------------------------------------

    # Filtrer sur une note moyenne minimale (vote_average)
    if "vote_average" in genre_filtered_df.columns:
        try:
            min_rating_input = input(
                "\nNote minimale sur 10 ? (appuie sur Entrée pour ignorer, ex: 6.5) : "
            ).strip()
            if min_rating_input:
                min_rating = float(min_rating_input)
                genre_filtered_df = genre_filtered_df[
                    (genre_filtered_df["vote_average"].notna())
                    & (genre_filtered_df["vote_average"] >= min_rating)
                ]
                print(f"Films après filtre sur la note >= {min_rating} : {len(genre_filtered_df)}")
        except ValueError:
            print("Saisie de note invalide, aucun filtre de note appliqué.")

    # Filtrer sur la durée (runtime) : par exemple, éviter les très longs si l'utilisateur le souhaite
    if "runtime" in genre_filtered_df.columns:
        try:
            max_runtime_input = input(
                "\nDurée max en minutes ? (appuie sur Entrée pour ignorer, ex: 120) : "
            ).strip()
            if max_runtime_input:
                max_runtime = int(max_runtime_input)
                genre_filtered_df = genre_filtered_df[
                    (genre_filtered_df["runtime"].notna())
                    & (genre_filtered_df["runtime"] <= max_runtime)
                ]
                print(f"Films après filtre sur la durée <= {max_runtime} min : {len(genre_filtered_df)}")
        except ValueError:
            print("Saisie de durée invalide, aucun filtre de durée appliqué.")

    if genre_filtered_df.empty:
        print("\nAucun film ne correspond à ces critères additionnels.")
        return

    # ---------------------------------------------------
    # Étape 4 : tri par popularité et sélection finale
    # ---------------------------------------------------

    # On enlève les films sans popularité définie
    final_df = genre_filtered_df[genre_filtered_df["popularity"].notna()].copy()

    if final_df.empty:
        print("\nAucun film avec une popularité définie après filtrage.")
        return

    # Tri décroissant par popularité
    final_df = final_df.sort_values(by="popularity", ascending=False)

    # Limiter au top_n
    final_df = final_df.head(top_n)

    # ---------------------------------------------------
    # Étape 5 : affichage des recommandations
    # ---------------------------------------------------

    print(f"\n=== TOP {len(final_df)} FILMS RECOMMANDÉS ===\n")

    for _, row in final_df.iterrows():
        title = row.get("title", "Titre inconnu")
        original_title = row.get("original_title", "")
        release_date = row.get("release_date", "Date inconnue")
        popularity = row.get("popularity", "N/A")
        rating = row.get("vote_average", "N/A")
        runtime = row.get("runtime", "N/A")

        # On affiche le titre + quelques infos utiles
        print(f"- {title} "
              f"{f'({original_title})' if original_title and original_title != title else ''}")
        print(f"  Date de sortie : {release_date}")
        print(f"  Popularité : {popularity} | Note : {rating} | Durée : {runtime} min")
        print()

    return final_df


### 3.4. Lancer le système intéractif

In [8]:
# Il est possible de lire les différentes options possibles par catégorie avant de faire son choix, l'algorithme les écrit en bas de cette cellule !

if __name__ == "__main__":
    recommend_movies(df)

Langues disponibles dans le dataset :
aa, ab, af, ak, am, an, ar, as, av, ay, az, ba, be, bg, bi, bm, bn, bo, br, bs, ca, ce, ch, cn, co, cr, cs, cv, cy, da, de, dv, dz, ee, el, en, eo, es, et, eu, fa, ff, fi, fj, fo, fr, fy, ga, gd, gl, gn, gu, gv, ha, he, hi, ho, hr, ht, hu, hy, hz, ia, id, ie, ig, ik, io, is, it, iu, ja, jv, ka, kg, ki, kj, kk, kl, km, kn, ko, ks, ku, kv, kw, ky, la, lb, lg, li, ln, lo, lt, lv, mg, mh, mi, mk, ml, mn, mo, mr, ms, mt, my, nb, nd, ne, ng, nl, nn, no, nr, nv, ny, oc, oj, om, or, os, pa, pl, ps, pt, qu, rm, rn, ro, ru, rw, sa, sc, sd, se, sg, sh, si, sk, sl, sm, sn, so, sq, sr, ss, st, su, sv, sw, ta, te, tg, th, ti, tk, tl, tn, to, tr, ts, tt, tw, ty, ug, uk, ur, uz, ve, vi, wo, xh, xx, yi, yo, za, zh, zu

Nombre de films trouvés pour la langue 'an': 2

Genres disponibles pour cette langue :
documentary

Films après filtre sur les genres : 1

=== TOP 1 FILMS RECOMMANDÉS ===

- 21.000 PALABRAS (Un capezuto e dos collons) 
  Date de sortie : 2025-06-26
 